In [29]:
import types, sys, IPython, traceback, ipykernel.ipkernel, traitlets, operator, toolz.curried as toolz, tokenize, io, textwrap

ip = IPython.get_ipython()

In [30]:

ip = get_ipython()

In [31]:
    def get_inline_expression(string, pos):
        try:
            line, offset = IPython.utils.tokenutil.line_at_cursor(string, pos)
        except tokenize.TokenError: return 
        inline = True
        snippet, token, line = '', '`', ''
        while (offset < pos) and line:
            inline = not inline
            snippet, token, line = line.partition('`')
            offset += len(snippet) + len(token)
        return inline and snippet or None

In [32]:
    class PidginKernel(ipykernel.ipkernel.IPythonKernel):        
        def do_inspect(self, code, cursor_pos, detail_level=0):
            inspect = False
            if (cursor_pos-2)>0 and cursor_pos-1 <= len(code) :
                if code[cursor_pos-1] == '?':
                    inspect = True
                    if code[cursor_pos-2] == '?':
                        detail_level = 1

            if not inspect and '`' in code:
                inline = get_inline_expression(code, cursor_pos)
                self.log.error((11,inline))
                if inline:
                    result = {}
                    self.log.error(result)
                    return dict(result, found=True, status='ok')

            result = ipykernel.ipkernel.IPythonKernel.do_inspect(self, code, cursor_pos, detail_level)
            
            if not result['found']:
                result['found'] = True
                result['data'].update({'text/markdown': code})
            return result

In [27]:
    original_methods = {}
    def load_ipython_extension(ip):
        global original_methods
        original_methods['do_inspect'] = original_methods.get(
            'do_inspect', ip.kernel.do_inspect)
        ip.kernel.do_inspect = types.MethodType(PidginKernel.do_inspect, ip.kernel)

    def unload_ipython_extension(ip): 
        ip.kernel.__dict__.update(original_methods)

In [28]:
    if __name__ == '__main__':
        #!ipython -m pytest -- shell.ipynb
        load_ipython_extension(get_ipython())